<h1>Utilise le cnn v1 et affiche les résultats</h1>


In [2]:
"""
    Use the CNN v1 model in order to make predictions for each machine (on files in data_png_cnn1/[machine]/png_test)
    from: https://keras.io/examples/vision/image_classification_from_scratch/
"""

import sys
import datetime
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from keras.preprocessing import image
from keras.models import load_model

# add parent folder to path 
from pathlib import Path
path = Path(os.getcwd())
str_parent = str(path.parent.absolute())
if not str_parent in sys.path:
    sys.path.append(str_parent)
import utils 

print(utils.list_datasets)

['valve', 'slider', 'pump', 'ToyCar', 'fan', 'ToyConveyor']


In [3]:

image_size = (333, 216)
cutoff = 0.9 # if score > cutoff, the machine predicted as "normal" otherwise as "anomaly"
indiceFile = 0
name_csv_logs = 'cnn_results.csv'
df_logs_h = pd.DataFrame(columns=['machine', 'file', 'correctPrediction', 'score'])
df_logs_h.to_csv(name_csv_logs, header = True, index = False)


In [4]:
def predict_one_machine(machine_folder):
    png_tests_folder = '../../_data_png_cnn1/' + machine_folder + '/png_test/'
    model_folder = '../../_classifiers_cnn1/cnn_' + machine_folder + '.h5'
    # renit the result dataframe (store with files tested and scores of the model)
    df_result = pd.DataFrame(columns=['file', 'score'])
    # load the model from disk
    model = load_model(model_folder)
    wavfiles = [f for f in listdir(png_tests_folder) if isfile(join(png_tests_folder, f))]
    for nameFilePngTotest in wavfiles:
        if nameFilePngTotest[-4:] != '.png': # ignore non .png files
            continue
        # print('test nameFilePngTotest: ', nameFilePngTotest)
        test_image = image.load_img(png_tests_folder + nameFilePngTotest, target_size = image_size) 
        img_array = image.img_to_array(test_image)
        img_array = np.expand_dims(test_image, axis = 0)
        predictions = model.predict(img_array)
        score = predictions[0]
        df_result = df_result.append({'file': nameFilePngTotest, 'score': score}, ignore_index=True)

    df_result = df_result.sort_values(by = ['score'], ascending = False)
    countPrediction = 0
    countCorrectPrediction = 0
    for index, row in df_result.iterrows():
        countPrediction += 1
        arrName = row['file'].split("_") # anomaly_id_00_00000001.wav
        classPrefix = arrName[0] # 'normal' or 'anomaly'
        isNormalPredict = 1 if row['score'] > cutoff else 0
        isNormalReal = 1 if classPrefix == "normal" else 0
        if isNormalReal == isNormalPredict:
            countCorrectPrediction += 1
        # if countPrediction % 20 == 0: # display result predictions (1 of 20)
        correctPrediction = 'OK' if isNormalReal == isNormalPredict else 'NOK'
        # print("file %s predict %s (score: %.2f%%)." % (row['file'], correctPrediction, 100 * row['score']))
        df_logs = pd.read_csv(name_csv_logs)
        new_row = {'machine': machine_folder, 'file': row['file'], 'correctPrediction': correctPrediction, 'score': row['score'][0]}
        df_logs = df_logs.append(new_row, ignore_index=True)
        df_logs.to_csv(name_csv_logs, header = True, index = False)
        
    # display general result for the machine (correct predictions / total of préeditions)
    print("****** %s: accuracy:  %.2f (%i fichiers)" % (machine_folder, countCorrectPrediction / len(df_result), countPrediction))

In [5]:
for machine_folder in utils.list_datasets: 
    predict_one_machine(machine_folder)
print('See the file: ', name_csv_logs, ' for details')
# See the file:  cnn_results.csv  for details
# ****** valve: accuracy:  0.92 (177 fichiers)
# ****** slider: accuracy:  0.96 (180 fichiers)
# ****** pump: accuracy:  0.83 (184 fichiers)
# ****** ToyCar: accuracy:  0.90 (195 fichiers)
# ****** fan: accuracy:  0.93 (183 fichiers)
# ****** ToyConveyor: accuracy:  0.50 (197 fichiers)
# See the file:  cnn_results.csv  for details

#2 demo
# ****** valve: accuracy:  0.63 (177 fichiers)
# ****** slider: accuracy:  0.97 (180 fichiers)
# ****** pump: accuracy:  0.51 (184 fichiers)
# ****** ToyCar: accuracy:  0.53 (195 fichiers)
# ****** fan: accuracy:  0.75 (183 fichiers)
# ****** ToyConveyor: accuracy:  0.50 (197 fichiers)


****** valve: accuracy:  0.63 (177 fichiers)
****** slider: accuracy:  0.97 (180 fichiers)
****** pump: accuracy:  0.51 (184 fichiers)
****** ToyCar: accuracy:  0.53 (195 fichiers)
****** fan: accuracy:  0.75 (183 fichiers)
****** ToyConveyor: accuracy:  0.50 (197 fichiers)
See the file:  cnn_results.csv  for details
